# MNIST - CNN Approach

In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
## Creation of helper functions
## Helper funtion will help initialize the weights, biases, conv2D, Pooling

# Init Weights

def InitWeights(shape):
    init_rand_dist = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(init_rand_dist)

In [5]:
# Init Bias (Same as InitWeights but for bias)

def InitBias(shape):
    init_bias_vals = tf.constant(0.1, shape = shape)
    return tf.Variable(init_bias_vals)

In [6]:
# Convenience function that helps create Conv2D, it takes in an input tensor and a kernel/filter and performs convolution on it

def conv2d(x, W):
    # x --> [batch, h, w, channels]
    # W --> [filter h, filter w, channel in, channel out]
    return tf.nn.conv2d(x, W, strides = [1, 1, 1, 1], padding = 'SAME')

In [7]:
# Pooling

def max_pool_2x2(x):
    # x --> [batch, h, w, channels]
    return tf.nn.max_pool(x, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')

## Now we can just call these convenience functions instead of calling the entire thing over and over again

In [8]:
# Funtions to actually create the layers

# Convolutional layer

def convolution_layer(input_x, shape):
    W = InitWeights(shape)
    b = InitBias([shape[3]])
    return tf.nn.relu(conv2d(input_x,  W) + b)

In [9]:
# Fully Connected layer

def full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = InitWeights([input_size, size])
    b = InitBias([size])
    return tf.matmul(input_layer, W) + b

In [10]:
# Creating Placeholders

x = tf.placeholder(tf.float32, shape = [None, 784])

In [11]:
y_true = tf.placeholder(tf.float32, shape = [None, 10])

In [12]:
# Creating the layers

x_image = tf.reshape(x, [-1, 28, 28, 1])

In [13]:
conv_1 = convolution_layer(x_image, shape = [5, 5, 1, 32]) # [5x5, 1 channel, 32 features(output channels)]
conv_1_pool = max_pool_2x2(conv_1)

In [14]:
conv_2 = convolution_layer(conv_1_pool, shape = [5, 5, 32, 64])
conv_2_pool = max_pool_2x2(conv_2)

In [15]:
# Flattening

conv_2_flat = tf.reshape(conv_2_pool, [-1, 7 * 7 * 64])
full_lay_1 = tf.nn.relu(full_layer(conv_2_flat, 1024))

In [16]:
# Adding Dropout

hold_prob = tf.placeholder(tf.float32)
full_1_dropout = tf.nn.dropout(full_lay_1, keep_prob = hold_prob)

In [17]:
y_pred = full_layer(full_1_dropout, 10)

## To create a loss function, an optimizer, initialize the variables and then run the session

In [18]:
# Loss Function

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = y_true, logits = y_pred))

In [19]:
# Optimizer

optimizer = tf.train.AdamOptimizer(learning_rate = 0.001)
train = optimizer.minimize(cross_entropy)

In [20]:
init = tf.global_variables_initializer()

In [ ]:
# Defining our session

steps = 100

with tf.Session() as sess:
    
    sess.run(init)
    
    for i in range(steps):
        
        batch_x, batch_y = mnist.train.next_batch(50)
        
        sess.run(train, feed_dict = {x : batch_x, y_true : batch_y, hold_prob : 0.5})
        
        if i%100 == 0: # Every 100 steps report the accuracy back
            
            print("On Step: {}".format(i))
            print('Accuracy: {}')
            matches = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
            
            # Casting the booleans into float
            
            acc = tf.reduce_mean(tf.cast(matches, tf.float32))
            
            print(sess.run(acc, feed_dict = {x : mnist.test.images, y_true : mnist.test.labels, hold_prob : 1.0}))
            print('\n')

On Step: 0
Accuracy: {}


# Conclusion: - 

## We get an accuracy of 98.84%